In [1]:
!pip -q install openai langchain==0.0.99rc0

## Basics with OpenAI API

In [ ]:
import os



os.environ['OPENAI_API_KEY'] = 'your-api-key'

In [3]:
pip install openai==0.28

In [4]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]
)

In [5]:
response

<OpenAIObject chat.completion id=chatcmpl-9NcmDv0NbVD2LCyXJluLPUFkNz3QJ at 0x7d2e0f24f970> JSON: {
  "id": "chatcmpl-9NcmDv0NbVD2LCyXJluLPUFkNz3QJ",
  "object": "chat.completion",
  "created": 1715417629,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! I am a helpful assistant here to provide you with assistance and information on a wide range of topics. How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 29,
    "total_tokens": 54
  },
  "system_fingerprint": null
}

#### Introduction to Chat Markup Language (ChatML)

**Chat Markup Language (ChatML)** was introduced by OpenAI on March 1, 2023, as part of the announcement of the ChatGPT and Whisper APIs. **ChatML is designed to bring structure to the format of user input data. It is used to make explicit to the model the source of each piece of text, showing the boundary between human and AI text. This helps to mitigate and eventually solve injections, as the model can tell which instructions come from the developer, the user, or its own input**.

#### Recent Advancements in ChatML

ChatML has been designed to accommodate a payload that is currently only text. However, OpenAI foresees the introduction of other data types in the future. This is in line with the notion of Large Foundation Models soon starting to combine text, images, sound, etc. This multi-modal approach will facilitate a more comprehensive interaction with AI models.

#### Impact on OpenAI APIs and Systems

The introduction of ChatML implies that systems making use of OpenAI APIs will have to structure their input in the ChatML format of messages for the system, user, and assistant. This will require adaptation from Autonomous Agent frameworks and Prompt Chaining IDEs. However, the addition of Function Calling was well received, and the imposing of ChatML could also have its benefits.

#### Future of ChatML

OpenAI continues to improve the GPT-35-Turbo and the Chat Markup Language used with the models will continue to evolve in the future. As the language matures, it is expected to offer more features and capabilities, further enhancing the interaction between users and AI models.

#### Conclusion

ChatML is a significant advancement in the field of AI, providing a structured format for user input data and paving the way for multi-modal interactions with AI models. As it continues to evolve, it is expected to play a crucial role in the development of AI applications and systems.

### Chat Markup Language - token system

```markdown
<|im_start|>system
You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.
Knowledge cutoff: 2021-09-01
Current date: 2023-03-01<|im_end|>
<|im_start|>user
How are you<|im_end|>
<|im_start|>assistant
I am doing well!<|im_end|>
<|im_start|>user
How are you now?<|im_end|>
```

```
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
```

In [9]:
messages=[
        {"role": "system", "content": "You are a helpful assistant named Kate."},
        {"role": "user", "content": "Hello what kind of assistant are you?"},
    ]

In [10]:
conversation_total_tokens = 0

while True:
    message = input("Human: ")
    if message=='exit':
        print(f"{conversation_total_tokens} tokens used in total in this conversation")
        break
    if message:
        messages.append(
            {"role": "user", "content": message},
        )
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo", messages=messages
        )

    reply = response.choices[0].message.content
    total_tokens = response.usage['total_tokens']
    conversation_total_tokens += total_tokens
    print(f"ChatGPT: {reply} \n {total_tokens} tokens used")
    messages.append({"role": "assistant", "content": reply})

Human: Who is girlfriend of Bradley Cooper?
ChatGPT: Bradley Cooper's girlfriend is currently Russian model Irina Shayk. 
 52 tokens used
Human: tell me something about yourself
ChatGPT: I am Kate, a helpful assistant here to provide you with information and assistance to the best of my abilities. Feel free to ask me anything you'd like to know! 
 99 tokens used
Human: What about Gigi Hadid?
ChatGPT: Gigi Hadid is an American supermodel who has gained widespread recognition for her modeling work with various high-profile fashion brands and her appearances on runway shows, magazine covers, and advertising campaigns. She is known for her striking beauty, tall stature, and successful career in the fashion industry. 
 171 tokens used
Human: Exit
ChatGPT: Goodbye! If you have any more questions in the future, feel free to ask. Have a great day! 
 203 tokens used
Human: exit
525 tokens used in total in this conversation


## ChatGPT with LangChain

In [11]:
!pip show langchain

Name: langchain
Version: 0.0.99rc0
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, aleph-alpha-client, dataclasses-json, deeplake, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [13]:
from langchain import PromptTemplate, LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI, OpenAIChat

In [12]:
prefix_messages = [{"role": "system", "content": "You are a helpful history professor named Kate."}]


In [14]:
## old way
# llm = OpenAI(model_name="text-davinci-003",
#              temperature=0, )

## New way
llm = OpenAIChat(model_name='gpt-3.5-turbo',
             temperature=0,
             prefix_messages=prefix_messages,
             max_tokens = 256)

In [15]:

template = """Take the following question: {user_input}

Answer it in an informative and intersting but conscise way for someone who is new to this topic."""

prompt = PromptTemplate(template=template,
                        input_variables=["user_input"])


In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "When was Marcus Aurelius the emperor of Rome?"

llm_chain.run(user_input)

'Marcus Aurelius was the emperor of Rome from 161 to 180 AD. He is often remembered as one of the "Five Good Emperors" of Rome, known for his philosophical writings and leadership during a time of great challenges for the Roman Empire. Marcus Aurelius\' reign was marked by wars on the northern frontier, the Antonine Plague, and political unrest, yet he is admired for his Stoic philosophy and commitment to duty.'

In [17]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "Who was Marcus Aurelius married to?"

llm_chain.run(user_input)

"Marcus Aurelius was married to Faustina the Younger, who was the daughter of Roman Emperor Antoninus Pius. Their marriage was arranged for political reasons, as was common among Roman aristocracy. Faustina was known for her beauty and intelligence, and she played a significant role in Marcus Aurelius' life, accompanying him on military campaigns and supporting him in his duties as emperor. Their marriage produced several children, including Commodus, who would later succeed Marcus Aurelius as emperor."